In [1]:
!pip install azure-storage-blob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.5 MB/s eta 0:00:00


In [2]:
# import Librairies
import pandas as pd
import numpy as np
import json
import requests
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from io import StringIO

In [3]:
URL = "https://data.cityofnewyork.us/resource/erm2-nwe9.csv"

In [4]:
limit = 10000
URL2 = f'{URL}?$limit={limit}'

In [5]:
df_raw = pd.read_csv(URL2)
df_raw.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location
0,60774651,2024-04-05T01:51:05.000,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10453.0,2244 CRESTON AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.856808,-73.901500,"\n, \n(40.856807704117266, -73.90149969181229)"
1,60778622,2024-04-05T01:50:41.000,NaN,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,10453.0,1887 HARRISON AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.852285,-73.911888,"\n, \n(40.852284600296855, -73.91188784004191)"
2,60778616,2024-04-05T01:50:22.000,NaN,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11235.0,2556 EAST 12 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.588496,-73.958386,"\n, \n(40.588495693409605, -73.95838622625635)"
3,60777468,2024-04-05T01:50:00.000,NaN,DSNY,Department of Sanitation,Illegal Dumping,Removal Request,Street,10026.0,180 ST NICHOLAS AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.805904,-73.952800,"\n, \n(40.805903709811055, -73.95279950931113)"
4,60769380,2024-04-05T01:49:34.000,NaN,DOT,Department of Transportation,Street Condition,Defective Hardware,Street,10026.0,24 WEST 117 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.801845,-73.947186,"\n, \n(40.8018445700123, -73.94718577356464)"


In [6]:
df_raw.columns

Index(['unique_key', 'created_date', 'closed_date', 'agency', 'agency_name',
       'complaint_type', 'descriptor', 'location_type', 'incident_zip',
       'incident_address', 'street_name', 'cross_street_1', 'cross_street_2',
       'intersection_street_1', 'intersection_street_2', 'address_type',
       'city', 'landmark', 'facility_type', 'status', 'due_date',
       'resolution_description', 'resolution_action_updated_date',
       'community_board', 'bbl', 'borough', 'x_coordinate_state_plane',
       'y_coordinate_state_plane', 'open_data_channel_type',
       'park_facility_name', 'park_borough', 'vehicle_type',
       'taxi_company_borough', 'taxi_pick_up_location', 'bridge_highway_name',
       'bridge_highway_direction', 'road_ramp', 'bridge_highway_segment',
       'latitude', 'longitude', 'location'],
      dtype='object')

In [7]:
df_raw.shape

(10000, 41)

In [8]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 41 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   unique_key                      10000 non-null  int64  
 1   created_date                    10000 non-null  object 
 2   closed_date                     4744 non-null   object 
 3   agency                          10000 non-null  object 
 4   agency_name                     10000 non-null  object 
 5   complaint_type                  10000 non-null  object 
 6   descriptor                      9889 non-null   object 
 7   location_type                   8459 non-null   object 
 8   incident_zip                    9827 non-null   float64
 9   incident_address                9328 non-null   object 
 10  street_name                     9328 non-null   object 
 11  cross_street_1                  6482 non-null   object 
 12  cross_street_2                  6

In [9]:
df_raw.dropna()
df_raw.shape

(10000, 41)

In [10]:
df_cleaned = df_raw.copy()

In [11]:
df_cleaned = df_raw.drop(columns = ['road_ramp'])

In [12]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 40 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   unique_key                      10000 non-null  int64  
 1   created_date                    10000 non-null  object 
 2   closed_date                     4744 non-null   object 
 3   agency                          10000 non-null  object 
 4   agency_name                     10000 non-null  object 
 5   complaint_type                  10000 non-null  object 
 6   descriptor                      9889 non-null   object 
 7   location_type                   8459 non-null   object 
 8   incident_zip                    9827 non-null   float64
 9   incident_address                9328 non-null   object 
 10  street_name                     9328 non-null   object 
 11  cross_street_1                  6482 non-null   object 
 12  cross_street_2                  6

In [14]:
from io import BytesIO, StringIO

# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

In [15]:
#specify the path to your JSON configuration file
config_file_path = 'config.json'

#Load the JSON configuration file
with open(config_file_path,'r') as config_file:
    config=json.load(config_file)

#Print the configuration
#Connection_STRING = config["connectionString"]

In [19]:
#Create Azure account, Azure storage
#Create container, copy access keys
CONNECTION_STRING_AZURE_STORAGE = config['connectionString']
CONTAINER_AZURE = 'homework1'

In [21]:
blob_name = "homework1_311_20240406.csv"

# Convert DataFrame to CSV
output = StringIO()
df_raw.to_csv(output, index=False)
data = output.getvalue()
output.close()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get a blob client using the container name and blob name
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

# Upload the CSV data
blob_client.upload_blob(data, overwrite=True)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")

Uploaded homework1_311_20240406.csv to Azure Blob Storage in container homework1.
